In [9]:
# ===========================
# SmartBiz Dashboard
# Author: Sudhama Mattegunta
# Purpose: Customer Analytics, Transactions Risk, Inventory Trends, Feedback Sentiment
# Features: Dynamic input via ipywidgets (CSV, Excel, Database), Interactive visualizations
# ===========================

# 0️⃣ Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
from ipywidgets import interact, widgets
from IPython.display import display
import sqlite3
import os
%matplotlib inline

sns.set(style="whitegrid")

# ===========================
# 1️⃣ Load Data Function (Dynamic)
# ===========================
def load_data(source_type, file_path=None):
    if source_type.lower() == 'database':
        conn = sqlite3.connect(file_path if file_path else 'smartbiz.db')
        customers = pd.read_sql_query('SELECT * FROM customers', conn)
        transactions = pd.read_sql_query('SELECT * FROM transactions', conn)
        inventory = pd.read_sql_query('SELECT * FROM inventory', conn)
        feedback = pd.read_sql_query('SELECT * FROM feedback', conn)
        conn.close()
    else:
        if not file_path or not os.path.exists(file_path):
            print("File path invalid!")
            return pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
        ext = file_path.split('.')[-1].lower()
        if ext == 'csv':
            customers = pd.read_csv(file_path)
            transactions = pd.read_csv(file_path.replace('customers','transactions'))
            inventory = pd.read_csv(file_path.replace('customers','inventory'))
            feedback = pd.read_csv(file_path.replace('customers','feedback'))
        elif ext in ['xls','xlsx']:
            customers = pd.read_excel(file_path, sheet_name='customers')
            transactions = pd.read_excel(file_path, sheet_name='transactions')
            inventory = pd.read_excel(file_path, sheet_name='inventory')
            feedback = pd.read_excel(file_path, sheet_name='feedback')
        else:
            print("Unsupported file type!")
            return pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

    # Inventory moving avg
    if 'weekly_sales' not in inventory.columns:
        inventory['weekly_sales'] = [0]*len(inventory)
    inventory['Moving_Avg'] = inventory['weekly_sales'].rolling(window=3,min_periods=1).mean()

    # Feedback sentiment
    feedback['Sentiment'] = feedback['review_text'].apply(lambda x: 'Positive' if TextBlob(x).sentiment.polarity>0 else 'Negative')

    # Customer analytics
    today = pd.to_datetime('2025-09-13')
    customers['Recency'] = (today - pd.to_datetime(customers['last_purchase_date'])).dt.days
    customers['Segment'] = pd.cut(customers['total_spent'], bins=[0,500,1500,10000], labels=['Low','Medium','High'])
    customers['Churn_Risk'] = customers['Recency'].apply(lambda x: 'High' if x>180 else 'Low')

    # Transaction risk
    transactions['Risk_Flag'] = transactions['amount'].apply(lambda x: 'High' if x>50000 else 'Normal')

    return customers, transactions, inventory, feedback

# ===========================
# 2️⃣ Dashboard Function
# ===========================
def show_dashboard(segment, source_type, file_path=None):
    customers, transactions, inventory, feedback = load_data(source_type, file_path)
    if customers.empty:
        print("No data available.")
        return

    # Segment filter
    cust_df = customers if segment=='All' else customers[customers['Segment']==segment]

    print("=== Customers ===")
    display(cust_df)

    print("=== High-Risk Transactions ===")
    display(transactions[transactions['Risk_Flag']=='High'])

    # Inventory plot
    plt.figure(figsize=(8,4))
    sns.lineplot(data=inventory, x='product_name', y='Moving_Avg', marker='o', color='navy')
    plt.title("Inventory Moving Average per Product", fontsize=14)
    plt.xlabel("Product")
    plt.ylabel("Moving Avg Sales")
    plt.show()

    # Feedback sentiment
    plt.figure(figsize=(6,4))
    sns.countplot(x='Sentiment', data=feedback, hue='Sentiment', dodge=False, palette=['lightgreen','salmon'])
    plt.title("Customer Feedback Sentiment", fontsize=14)
    plt.legend([],[], frameon=False)
    plt.show()

    # Interactive filters
    def show_customers_widget(seg):
        display(customers[customers['Segment']==seg])
    interact(show_customers_widget, seg=['Low','Medium','High'])

    def churn_pie(days):
        temp = customers.copy()
        temp['Churn_Risk'] = temp['Recency'].apply(lambda x: 'High' if x>days else 'Low')
        temp['Churn_Risk'].value_counts().plot(kind='pie', autopct='%1.1f%%', colors=['lightcoral','lightgreen'])
        plt.title(f"Churn Risk > {days} Days", fontsize=14)
        plt.ylabel("")
        plt.show()
    interact(churn_pie, days=(30,365,30))

    def inventory_trend(product_name):
        temp = inventory[inventory['product_name']==product_name]
        sns.lineplot(data=temp, x='product_name', y='Moving_Avg', marker='o', color='blue')
        plt.title(f"{product_name} – Inventory Trend", fontsize=14)
        plt.xlabel("Product")
        plt.ylabel("Moving Avg")
        plt.show()
    interact(inventory_trend, product_name=inventory['product_name'].tolist())

    def show_risk(threshold):
        temp = transactions[transactions['amount']>threshold]
        print(f"Transactions above {threshold}:")
        display(temp)
    interact(show_risk, threshold=(1000,100000,5000))

    def show_feedback_widget(sentiment):
        display(feedback[feedback['Sentiment']==sentiment])
    interact(show_feedback_widget, sentiment=['Positive','Negative'])

# ===========================
# 3️⃣ Interactive Input Widgets
# ===========================
source_widget = widgets.Dropdown(
    options=['Database', 'CSV', 'Excel'],
    description='Data Source:'
)

file_widget = widgets.Text(
    description='File Path:',
    placeholder='Enter file path if CSV/Excel'
)

segment_widget = widgets.Dropdown(
    options=['All','Low','Medium','High'],
    description='Segment:'
)

ui = widgets.VBox([source_widget, file_widget, segment_widget])

out = widgets.interactive_output(
    show_dashboard,
    {'segment': segment_widget,
     'source_type': source_widget,
     'file_path': file_widget}
)

display(ui, out)


Output()